In [1]:
import math

import numpy as np
import torch
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler

from typing import List

In [2]:
train_df, test_df = msrank_10k()

In [8]:
train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2.0,1,3,3,0,0,3,1.0,1.0,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.0
1,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.0,1,3,0,2,0,3,1.0,0.0,0.666667,...,45,3,1,124,3344,14,67,0,0,0.0
3,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.0
4,1.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,64,5,7,256,49697,1,13,0,0,0.0


In [5]:
train_df.shape, test_df.shape

((10000, 138), (10000, 138))

In [6]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features: int, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        # укажите архитектуру простой модели здесь
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )

    def forward(self, input_1: torch.Tensor) -> torch.Tensor:
        logits = self.model(input_1)
        return logits

In [7]:
def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))

In [3]:
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
query_ids_train = train_df[1].values.astype(int)

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
query_ids_test = test_df[1].values.astype(int)

In [36]:
query_ids_train.shape[0]

10000

In [33]:
_, last_indices = np.unique(query_ids_train, return_index=True)
last_indices

array([   0,   86,  192,  284,  404,  463,  508,  582,  605,  659,  831,
        955, 1032, 1109, 1417, 1512, 1638, 1743, 1970, 2051, 2069, 2130,
       2225, 2316, 2413, 2494, 2650, 2701, 2897, 3121, 3243, 3299, 3356,
       3508, 3597, 3675, 3816, 4049, 4279, 4479, 4596, 4671, 4955, 5088,
       5204, 5327, 5490, 5794, 5861, 5998, 6087, 6105, 6311, 6368, 6438,
       6514, 6573, 6715, 6802, 6941, 6987, 7267, 7287, 7411, 7496, 7604,
       7694, 7809, 7983, 8158, 8261, 8389, 8466, 8595, 8698, 8781, 8946,
       9013, 9159, 9213, 9290, 9357, 9428, 9584, 9717, 9824, 9876])

In [4]:
np.unique(query_ids_train)

array([   1,   16,   31,   46,   61,   76,   91,  106,  121,  136,  151,
        166,  181,  196,  211,  226,  241,  256,  271,  286,  301,  316,
        331,  346,  361,  376,  391,  406,  421,  436,  451,  466,  481,
        496,  511,  526,  541,  556,  571,  586,  601,  616,  631,  646,
        661,  676,  691,  706,  721,  736,  751,  766,  781,  796,  811,
        826,  841,  856,  871,  886,  901,  916,  931,  946,  961,  976,
        991, 1006, 1021, 1036, 1051, 1066, 1081, 1096, 1111, 1126, 1141,
       1156, 1171, 1186, 1201, 1216, 1231, 1246, 1261, 1276, 1291])

In [ ]:
_, last_indices = np.unique(query_ids_train, return_index=True)
last_indices = np.append(last_indices, query_ids_train.shape[0])
start = 0
scaler = StandardScaler()
normalized_data = []
for stop in last_indices[1:]:
    normalized_sample_by_query = scaler.fit_transform(X_train[start:stop, :])
    start = stop
    print(normalized_sample_by_query.shape)
    normalized_data.append(normalized_sample_by_query)

normalized_data = np.vstack(normalized_data)
    
normalized_data.shape

(86, 136)
(106, 136)
(92, 136)
(120, 136)
(59, 136)
(45, 136)
(74, 136)
(23, 136)
(54, 136)
(172, 136)
(124, 136)
(77, 136)
(77, 136)
(308, 136)
(95, 136)
(126, 136)
(105, 136)
(227, 136)
(81, 136)
(18, 136)
(61, 136)
(95, 136)
(91, 136)
(97, 136)
(81, 136)
(156, 136)
(51, 136)
(196, 136)
(224, 136)
(122, 136)
(56, 136)
(57, 136)
(152, 136)
(89, 136)
(78, 136)
(141, 136)
(233, 136)
(230, 136)
(200, 136)
(117, 136)
(75, 136)
(284, 136)
(133, 136)
(116, 136)
(123, 136)
(163, 136)
(304, 136)
(67, 136)
(137, 136)
(89, 136)
(18, 136)
(206, 136)
(57, 136)
(70, 136)
(76, 136)
(59, 136)
(142, 136)
(87, 136)
(139, 136)
(46, 136)
(280, 136)
(20, 136)
(124, 136)
(85, 136)
(108, 136)
(90, 136)
(115, 136)
(174, 136)
(175, 136)
(103, 136)
(128, 136)
(77, 136)
(129, 136)
(103, 136)
(83, 136)
(165, 136)
(67, 136)
(146, 136)
(54, 136)
(77, 136)
(67, 136)
(71, 136)
(156, 136)
(133, 136)
(107, 136)
(52, 136)
(124, 136)


(10000, 136)

In [24]:
X_train

array([[3., 3., 0., ..., 0., 0., 0.],
       [3., 0., 3., ..., 0., 0., 0.],
       [3., 0., 2., ..., 0., 0., 0.],
       ...,
       [2., 0., 2., ..., 0., 0., 0.],
       [2., 0., 1., ..., 0., 0., 0.],
       [2., 1., 1., ..., 0., 0., 0.]])

In [10]:
def scale_features_in_query_groups(inp_feat_array: np.ndarray,
                                    inp_query_ids: np.ndarray) -> np.ndarray:
    # допишите ваш код здесь
    scaler = StandardScaler()
    scaler.fit(inp_feat_array)
    return scaler.transform(inp_query_ids)

In [11]:
scale_features_in_query_groups(X_train, query_ids_train)

ValueError: Expected 2D array, got 1D array instead:
array=[1.000e+00 1.000e+00 1.000e+00 ... 1.291e+03 1.291e+03 1.291e+03].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
from solution import ListNet, Solution

sol = Solution()

In [1]:
! pip install tqdm